<h1>Sentiment Analyse Englisch</h1>

In [1]:
import csv
import numpy as np
import pandas as pd
import re
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import datetime
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel
import torch
import emoji

In [2]:
sid = SentimentIntensityAnalyzer()

<p>Laden des Datensatzes</p>

In [50]:
df = pd.read_csv("/Users/tobias/Dev/FOM/Master_Thesis/data/Final_EN/Dataset_Wirecard.csv", sep=";",
                 parse_dates=["created_at"])

<h2>BERTweet</h2>

In [199]:
#remove duplicate emojis
def remove_duplicate_emoji(orig_str):
    prev_emoji = None
    remove_duplicate_emoji = []
    for c in orig_str:
        if c in emoji.EMOJI_DATA:
            if prev_emoji == c:
                continue
            prev_emoji = c
        remove_duplicate_emoji.append(c)
    return "".join(remove_duplicate_emoji)

#remove all emojis
def de_emojify(text):
    regex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F" 
        u"\U0001F300-\U0001F5FF"
        u"\U0001F680-\U0001F6FF"
        u"\U0001F1E0-\U0001F1FF"
                           "]+", flags = re.UNICODE)
    return regex_pattern.sub(r'', text)

In [178]:
#from transformers import AutoTokenizer, AutoModelForSequenceClassification
model_name = "finiteautomata/bertweet-base-sentiment-analysis"

model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

In [280]:
#Zum Testen
for each in range(37400,37405):
    print(each)
    try:
        if len(emoji.demojize(df.iloc[each]["text"])) >= 425:
            print(each, " in if 1")
            if len(emoji.demojize(remove_duplicate_emoji(df.iloc[each]["text"]))) >= 425:
                print(each, " in if 2")
                if len(de_emojify(df.iloc[each]["text"])) >= 425:
                    print(each, " in if 3")
                    continue
                else:
                    print("Add ohne Emojis")
                    print(classifier(de_emojify(df.iloc[each]["text"]))[0]["label"])
            else:
                print("Add ohne doppelte Emojis")
                print(classifier(emoji.demojize(remove_duplicate_emoji(df.iloc[each]["text"])))[0]["label"])
        else:
            print("Add")
            print(classifier(emoji.demojize(df.iloc[each]["text"]))[0]["label"])
    except Exception:
        print(each, "NEU")

37400
Add
NEU
37401
Add
37401 NEU
37402
Add
NEU
37403
Add
NEU
37404
Add
NEG


In [281]:
#Produktiv
list_sentiment_bert = []
list_error_tweets = []
n = 500
for each in range(len(df)):
    if (each/n) == 1:
        print(each)
        n = n + 500
    try:
        if len(emoji.demojize(df.iloc[each]["text"])) >= 425:
            if len(emoji.demojize(remove_duplicate_emoji(df.iloc[each]["text"]))) >= 425:
                if len(de_emojify(df.iloc[each]["text"])) >= 425:
                    continue
                else:
                    list_sentiment_bert.append(classifier(de_emojify(df.iloc[each]["text"]))[0]["label"])
            else:
                list_sentiment_bert.append(classifier(emoji.demojize(remove_duplicate_emoji(df.iloc[each]["text"])))[0]["label"])
        else:
            list_sentiment_bert.append(classifier(emoji.demojize(df.iloc[each]["text"]))[0]["label"])
    except Exception:
        print(each)
        list_sentiment_bert.append("NEU")
        list_error_tweets.append(each)

500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000
20500
21000
21500
22000
22500
23000
23500
24000
24500
25000
25500
26000
26500
27000
27500
28000
28500
29000
29500
30000
30500
31000
31500
32000
32500
33000
33500
34000
34500
35000
35500
36000
36500
37000
37401
37500
38000
38500
39000
39500
40000
40500
41000
41500
42000
42500
43000
43500
44000
44500
45000
45500
46000
46500
47000
47500
48000
48500
49000
49500
50000
50500
51000
51500
52000
52500
53000
53500
54000
54500
55000
55500
56000
56500
57000
57500
58000
58500
59000
59500
60000
60500
61000
61500
62000
62500
63000
63500
64000
64500
65000
65500
65723
66000
66500
67000
67500
68000
68500
69000
69500
70000
70500
71000
71500
72000
72500
73000
73500
74000
74500
75000
75500
76000
76500
77000
77500
78000
78500
79000
79500
80000
80500
81000
81500
82000
82500
83000
83500
84000


In [282]:
print(len(df))
print(len(list_sentiment_bert))
print(len(list_error_tweets))

111199
111199
3


In [311]:
list_backup = list_sentiment_bert

In [310]:
#Check ob classifier bei tweets mit error doch funktioniert, wenn Sonderzeichen gelöscht werden.
for each in list_error_tweets:
    print(re.sub('[#|$|=|(|)|&|\r]', '', df.iloc[each]["text"]), "\n")
    print(len(re.sub('[#|$|=|(|)|&|\r]', '', df.iloc[each]["text"])), "\n")
    print(classifier(re.sub('[#|$|=|(|)|&|\r]', '', df.iloc[each]["text"])), "\n")
    print(classifier(re.sub('[#|$|=|(|)|&|\r]', '', df.iloc[each]["text"]))[0]["label"], "\n")

EU OPEN: Indices: Slightly softer. Eurostoxx 50 -0.2%Sectors: Health care leads, IT lagsMovers: ASM +7%, Carrefour +4%, Bayer +3%,  Wirecard +2.2%, UBS +1.8%, Michelin +1.8%, Nokia -10%, Sainsbury's -4.2%, Peugeot -2.4%, RBS -1.5%, CBK -1.1%, Barcs -1% 

252 

[{'label': 'NEU', 'score': 0.9029528498649597}] 

NEU 

 Update:  Back on Friday, June 12th, 2020, Mr. Sayyid Imrani, Customer Service Manager 610 420-4824 [ sayyid.imrani ] from Wirecard contacted 

141 

[{'label': 'NEU', 'score': 0.9708991050720215}] 

NEU 

WayFair-gt;Snopes-gt;Wirecard-gt;at Jack-gt;MSM Media-gt;1% CIA-gt;Soros-gt;House of Saud-gt;‘FED’ Central Bank-gt;Roths-gt;Hollywood-gt;Jeffrey Epstein Island-gt;George Floyd-gt;Kobe Bryant-gt;Aliens Exist-gt;HRC-gt;Blrock 1-gt;Slush-gt;Alice in Wonderland-gt;WikiLeaks-gt;Mass Ext. Events-gt;QAnon 

299 

[{'label': 'NEG', 'score': 0.9800840616226196}] 

NEG 



In [312]:
#Anpassung der Ergebnisse mit Modifizierten Texten
for each in list_error_tweets:
    list_sentiment_bert[each] = classifier(re.sub('[#|$|=|(|)|&|\r]', '', df.iloc[each]["text"]))[0]["label"]

In [313]:
#Überprüfen ob es funktioniert hat
for each in list_error_tweets:
    print(each)
    print(list_sentiment_bert[each])

37401
NEU
65723
NEU
99465
NEG


In [314]:
#Ergebnisse dem DF hinzufügen
df["sentiment_bert"] = list_sentiment_bert

In [315]:
df.head()

,conversation_id,author_id,created_at,tweet_id,lang,retweet_count,reply_count,like_count,quote_count,source,tweet_type,referenced_tweet_id,text,sentiment_bert
0,703142204341026818,349287204,2016-02-26 08:58:59+00:00,703142204341026818,en,0,0,1,0,Twitter Web Client,None,None,1/2 I see Wirecard is rebounding after CEO Mar...,NEU
1,695255175334924288,428593285,2016-02-04 14:38:45+00:00,695255175334924288,en,0,0,0,0,IFTTT,None,None,Thank you for following us Markus Braun!,POS
2,695237701386244098,95129352,2016-02-04 13:29:19+00:00,695237701386244098,en,1,0,1,0,Twitter Web Client,None,None,"Disrupt the Banks! (Jeff Stewart, Markus Braun...",NEG
3,718385114170966016,17616154,2016-04-08 10:28:52+00:00,718385114170966016,en,0,0,0,0,Twitter Web Client,None,None,WDI wirecard CEO Markus Braun on Q4 2015 Resul...,NEU
4,718384477215567873,546713,2016-04-08 10:26:20+00:00,718384477215567873,en,1,0,3,0,Twitter Web Client,None,None,Wirecard Analyst Call transcript is available ...,NEU


In [316]:
#Backup CSV erstellen
df.to_csv("Dataset_Bert.csv", sep=";", index=False)

<h2>VADER</h2>
<p>Die Werte sind nun alle zwischen -1 und 1. Die Einteilung nach POS, NEG und NEU sieht wie folgt aus: POS &gt= 0.05, -0.05 &lt NEU &gt 0.05 und NEG &lt= -0.05. <br> Das sind die offiziellen Parameter laut: <a href="https://github.com/cjhutto/vaderSentiment#about-the-scoring">Git Hup Repo</a> <br> Ich muss schauen, ob sich diese auch für mich anwenden lassen.</p>

In [426]:

list_sentiment_vader = []
n = 500
for i in range(len(df)):
    text_to_analyse = df.iloc[i]["text"]
    
    if (i/n) == 1:
        print(i)
        n = n + 500
    
    for each_emoji in emoji.distinct_emoji_list(df.iloc[i]["text"]):
        text_to_analyse = text_to_analyse.replace(each_emoji, re.sub('[_\r]', ' ', emoji.demojize(each_emoji, delimiters=(' ', ''))))
    list_sentiment_vader.append(sid.polarity_scores(text_to_analyse)["compound"])


500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000
20500
21000
21500
22000
22500
23000
23500
24000
24500
25000
25500
26000
26500
27000
27500
28000
28500
29000
29500
30000
30500
31000
31500
32000
32500
33000
33500
34000
34500
35000
35500
36000
36500
37000
37500
38000
38500
39000
39500
40000
40500
41000
41500
42000
42500
43000
43500
44000
44500
45000
45500
46000
46500
47000
47500
48000
48500
49000
49500
50000
50500
51000
51500
52000
52500
53000
53500
54000
54500
55000
55500
56000
56500
57000
57500
58000
58500
59000
59500
60000
60500
61000
61500
62000
62500
63000
63500
64000
64500
65000
65500
66000
66500
67000
67500
68000
68500
69000
69500
70000
70500
71000
71500
72000
72500
73000
73500
74000
74500
75000
75500
76000
76500
77000
77500
78000
78500
79000
79500
80000
80500
81000
81500
82000
82500
83000
83500
84000
84500
85000


In [428]:
print(len(df))
print(len(list_sentiment_vader))

111199
111199


In [431]:
list_sentiment_vader[0:10]

[0.0992, 0.4199, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.3382, -0.3382]

In [437]:
#Validierung eines Tweets, ob Emojis geändert wurden -> Ja
print(sid.polarity_scores(df.iloc[94]["text"]))
print(list_sentiment_vader[94])

test_text_analyse = df.iloc[94]["text"]
for each in emoji.distinct_emoji_list(df.iloc[94]["text"]):
    print(each)
    test_text_analyse = test_text_analyse.replace(each, re.sub('[_\r]', ' ', emoji.demojize(each, delimiters=(' ', ''))))
print(test_text_analyse, ": ", sid.polarity_scores(test_text_analyse)["compound"])



{'neg': 0.0, 'neu': 0.531, 'pos': 0.469, 'compound': 0.91}
0.9894
😊
 HAVE A BLESSED DAY. HOPE YOU WOULD HAVE A READ ON MY BLOG. THANK YOU VERY MUCH.  smiling face with smiling eyes smiling face with smiling eyes smiling face with smiling eyes :  0.9894


In [439]:
df["sentiment_vader"] = list_sentiment_vader

In [440]:
df.head()

,conversation_id,author_id,created_at,tweet_id,lang,retweet_count,reply_count,like_count,quote_count,source,tweet_type,referenced_tweet_id,text,sentiment_bert,sentiment_vader
0,703142204341026818,349287204,2016-02-26 08:58:59+00:00,703142204341026818,en,0,0,1,0,Twitter Web Client,None,None,1/2 I see Wirecard is rebounding after CEO Mar...,NEU,0.0992
1,695255175334924288,428593285,2016-02-04 14:38:45+00:00,695255175334924288,en,0,0,0,0,IFTTT,None,None,Thank you for following us Markus Braun!,POS,0.4199
2,695237701386244098,95129352,2016-02-04 13:29:19+00:00,695237701386244098,en,1,0,1,0,Twitter Web Client,None,None,"Disrupt the Banks! (Jeff Stewart, Markus Braun...",NEG,0.0000
3,718385114170966016,17616154,2016-04-08 10:28:52+00:00,718385114170966016,en,0,0,0,0,Twitter Web Client,None,None,WDI wirecard CEO Markus Braun on Q4 2015 Resul...,NEU,0.0000
4,718384477215567873,546713,2016-04-08 10:26:20+00:00,718384477215567873,en,1,0,3,0,Twitter Web Client,None,None,Wirecard Analyst Call transcript is available ...,NEU,0.0000


In [446]:
df.loc[(df.sentiment_vader < 0.05) & (df.sentiment_vader > -0.05), "sentiment_vader"].count()

37229

In [449]:
df.loc[df.sentiment_bert == "NEU", "sentiment_bert"].count()

60884

In [450]:
#Backup CSV erstellen
df.to_csv("Dataset_Bert_Vader.csv", sep=";", index=False)